# Template script for a strategy.
This includes
- Parsing the data from the Trader service into a pandas dataframe which is appropriate for creating technical indicators and producing trading signals
- Defining a standardized function to create all the technical indicators requried
- Defining a standardized function to create the signals across the time frame provided
- Defining a standardized function that returns the signals (if any) for the most recent time frame
- Defining a standardized function that returns the results of a backtest
- Defining a standardized function that returns the results of an optimization run

### 1. Parsing the data from the Trader service

In [1]:
import pandas as pd

def parse_data(json_data: str):
    df = pd.read_json(json_data)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df.sort_values(by='date', inplace=True)
    df['Close'] = df['adj_close']
    df['Open'] = df['adj_open']
    df['High'] = df['adj_high']
    df['Low'] = df['adj_low']
    df['Volume'] = df['volume']
    return df


### 2. Create the technical indicators

In [27]:
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator
from ta.trend import EMAIndicator, SMAIndicator, ADXIndicator, MACD
import numpy as np

# For this strategy template, we will use very conservative parameters. The idea is to find very rare but good signals, and then apply to thousands of stocks.

ema_window = 200
sma_window_slow = 20
sma_window_fast = 10
rsi_window = 14
bb_window = 20
bb_std = 2.5
adx_window = 14
macd_window_slow = 26
macd_window_fast = 12
macd_window_signal = 9
        
def add_trend_indicator(df: pd.DataFrame, rsi_buy_threshold = 50, rsi_sell_threshold = 50, trend_window = 9):
    """
    This function adds a trend indicator to the dataframe, called "Trend"
    - It returns 1 if there is an up-trend
    - It returns -1 if there is a down-trend
    - It returns 0 if there is no trend

    An up-trend is defined by the following conditions:
    - The fast SMA is above the slow SMA
    - The MACD signal line is above the MACD line
    - The RSI is above 50

    """

    df["Trend"] = 0

    df.loc[(df['SMA_fast'] > df['SMA_slow']) & (df['MACD_signal'] > df['MACD_line']) & (df['RSI'] > rsi_buy_threshold), 'Trend'] = 1
    df.loc[(df['SMA_fast'] < df['SMA_slow']) & (df['MACD_signal'] < df['MACD_line']) & (df['RSI'] < rsi_sell_threshold), 'Trend'] = -1

    # We will then smooth the trend indicator with a rolling mean
    df['Trend'] = df['Trend'].rolling(trend_window).mean()

    return df


def add_indicators(df: pd.DataFrame, 
                   ema_window=ema_window, 
                   sma_window_slow=sma_window_slow, 
                   sma_window_fast=sma_window_fast, 
                   rsi_window=rsi_window, 
                   bb_window=bb_window, 
                   bb_std=bb_std,
                   adx_window=adx_window,
                   macd_window_slow=macd_window_slow,
                   macd_window_fast=macd_window_fast,
                   macd_window_signal=macd_window_signal):
    
    df['BB_high'] = BollingerBands(df['Close'], window=bb_window, window_dev=bb_std).bollinger_hband()
    df['BB_low'] = BollingerBands(df['Close'], window=bb_window, window_dev=bb_std).bollinger_lband()
    df['BB_width'] = BollingerBands(df['Close'], window=bb_window, window_dev=bb_std).bollinger_wband()
    df['RSI'] = RSIIndicator(df['Close'], window=rsi_window).rsi()
    df['EMA'] = EMAIndicator(df['Close'], window=ema_window).ema_indicator()
    df['SMA_slow'] = SMAIndicator(df['Close'], window=sma_window_slow).sma_indicator()
    df['SMA_fast'] = SMAIndicator(df['Close'], window=sma_window_fast).sma_indicator()
    df['ADX_pos'] = ADXIndicator(df['High'],df['Low'], df['Close'],  window=adx_window).adx_pos()
    df['ADX_neg'] = ADXIndicator(df['High'],df['Low'], df['Close'],  window=adx_window).adx_neg()
    df['ADX_index'] = ADXIndicator(df['High'],df['Low'], df['Close'],  window=adx_window).adx()
    df['MACD_line'] = MACD(df['Close'],window_slow=macd_window_slow, window_fast=macd_window_fast, window_sign=macd_window_signal).macd()
    df['MACD_signal'] = MACD(df['Close'],window_slow=macd_window_slow, window_fast=macd_window_fast, window_sign=macd_window_signal).macd_signal()
    df['MACD_hist'] = MACD(df['Close'],window_slow=macd_window_slow, window_fast=macd_window_fast, window_sign=macd_window_signal).macd_diff()
    add_trend_indicator(df)


    return df



### 3. Create the signals
A signal needs to include all the information required to place an order:
- Type of trade: Buy or Sell
- Entry price limit
- Stop-loss
- Take-profit
- Expiry time of the order

In [33]:
def create_signals(df: pd.DataFrame):
    """
    This is a super conservative strategy looking for rare but good signals.
    We will createa a Buy signal when the following conditions are met:
    - There is a general uptrend in the market: The fast moving SMA is above the slow moving SMA and the MACD Signal is above the MACD Line 
    - And, the lastest closing price is below the lower Bollinger Band.
    We apply the converse logic for Sell signals.
    """
    df['Signal'] = ''
    
    df.loc[(df['Close'] < df['BB_low']) & (df['Trend'] > 0.2), 'Signal'] = 'Buy'
    df.loc[(df['Close'] > df['BB_high']) & (df['Trend'] < -0.2), 'Signal'] = 'Sell'
    # df.loc[(df['Close'] < df['BB_low']) & (df['SMA_slow'] > df['SMA_fast']), 'Signal'] = 'Buy'
    # df.loc[(df['Close'] > df['BB_high']) & (df['SMA_slow'] > df['SMA_fast']), 'Signal'] = 'Sell'
    # df.loc[(df['Close'] < df['BB_low']) & (df['ADX_trend'] > 0), 'Signal'] = 'Buy'
    # df.loc[(df['Close'] > df['BB_high']) & (df['ADX_trend'] < 0), 'Signal'] = 'Sell'
    # df.loc[(df['Close'] < df['BB_low']) & (df['SMA'] > df['EMA']) & (df['SMA2'] < df['SMA']), 'Signal'] = 'Buy'
    # df.loc[(df['Close'] > df['BB_high']) & (df['SMA'] < df['EMA'])& (df['SMA2'] > df['SMA']), 'Signal'] = 'Sell'
    # df.loc[(df['Close'] > df['BB_high']) & (df['RSI'] > 50) & (df['SMA'] < df['EMA']), 'Signal'] = 'Sell'
    # df.loc[(df['Close'] < df['BB_low']) & (df['RSI'] < 50) & (df['SMA'] > df['EMA']), 'Signal'] = 'Buy'
    return df

limit = 0.00
stop_loss = 0.10 # This is in fractions of the current closing price
take_profit_long = 2
take_profit_short = 0.5
quantity = 0.9

def process_buy(row: pd.Series):
    row['Limit']=row['Close'] * (1+limit)
    row['Stop_loss']=row['Close'] * (1-stop_loss)
    # row['Stop_loss']=row['Close'] - (row['BB_high'] - row['BB_low']) * stop_loss
    row['Quantity'] = quantity
    row['Take_profit'] = row['Close'] * (1 + take_profit_long)
    return row

def process_sell(row: pd.Series):
    row['Limit']=row['Close'] * (1-limit)
    row['Stop_loss']=row['Close'] * (1+stop_loss)
    # row['Stop_loss']=row['Close'] + (row['BB_high'] - row['BB_low']) * stop_loss
    row['Quantity'] = quantity
    row['Take_profit'] = row['Close'] * (1 - take_profit_short)
    return row

def add_trades(df: pd.DataFrame):
    df['Limit'] = None
    df['Stop_loss'] = None
    df['Quantity'] = None
    df['Take_profit'] = None
    df['Expiry'] = None
    
    dfSignal = df[df['Signal'] != '']

    for index, row in dfSignal.iterrows():
        if row['Signal'] == 'Buy':
            df.loc[index] = process_buy(row)
        elif row['Signal'] == 'Sell':
            df.loc[index] = process_sell(row)

    return df


### Intermezzo - We need a helper plot function
This is for testing purposes only

In [29]:
# import plotly.io as pio
import plotly.offline as pyo
# pio.renderers.default = 'vscode'

import plotly.graph_objects as go
from plotly.subplots import make_subplots

def add_buy_sell_points(df: pd.DataFrame):
    df['sell_dp'] = None
    df['buy_dp'] = None
    df['sell_sl'] = None
    df['buy_sl'] = None
    df.loc[df['Signal'] == 'Sell', 'sell_dp'] = df['Limit']
    df.loc[df['Signal'] == 'Buy', 'buy_dp'] = df['Limit']
    df.loc[df['Signal'] == 'Sell', 'sell_sl'] = df['Stop_loss']
    df.loc[df['Signal'] == 'Buy', 'buy_sl'] = df['Stop_loss']
    return df

def create_plot(df: pd.DataFrame):

    # dfpl = df[200:400].copy()
    dfpl = df.copy()
    dfpl = add_buy_sell_points(dfpl)
    
    #dfpl=dfpl.drop(columns=['level_0'])#!!!!!!!!!!
    #dfpl.reset_index(inplace=True)

    data = [go.Candlestick(x=dfpl.index,
                           open=dfpl['Open'],               
                           high=dfpl['High'],
                           low=dfpl['Low'],
                           close=dfpl['Close'], yaxis='y1'),
                           go.Scatter(x=dfpl.index, y=dfpl['EMA'], line=dict(color='orange', width=2), name="EMA", yaxis='y1'),
                           go.Scatter(x=dfpl.index, y=dfpl['SMA_slow'], line=dict(color='yellow', width=2), name="SMA Slow", yaxis='y1'),
                           go.Scatter(x=dfpl.index, y=dfpl['SMA_fast'], line=dict(color='yellow', width=2, dash='dash'), name="SMA Fast", yaxis='y1'),
                           go.Scatter(x=dfpl.index, y=dfpl['BB_low'], line=dict(color='blue', width=1), name="BBL", yaxis='y1'),
                           go.Scatter(x=dfpl.index, y=dfpl['BB_high'], line=dict(color='blue', width=1), name="BBH", yaxis='y1'),
                           go.Scatter(x=dfpl.index, y=dfpl['sell_dp'], mode="markers",marker=dict(size=10, color="darkorange", symbol="triangle-down"),name="Sell", hovertemplate='Sell: %{y:.2f}', yaxis='y1'),
                           go.Scatter(x=dfpl.index, y=dfpl['buy_dp'], mode="markers",marker=dict(size=10, color="darkgreen", symbol="triangle-up"),name="Buy", hovertemplate='Buy: %{y:.2f}', yaxis='y1'),
                           go.Scatter(x=dfpl.index, y=dfpl['Stop_loss'], mode="markers",marker=dict(size=10, color="red", symbol="square-open"),name="Stop loss", hovertemplate='Stop loss: %{y:.2f}', yaxis='y1'),
                           go.Scatter(x=dfpl.index, y=dfpl['MACD_line'], line=dict(color='black', width=1), name="MACD", yaxis='y3'),
                           go.Scatter(x=dfpl.index, y=dfpl['MACD_signal'], line=dict(color='red', width=1), name="MACD Signal", yaxis='y3'),
                           go.Scatter(x=dfpl.index, y=dfpl['Trend'], line=dict(color='blue', width=1), name="Trend", yaxis='y2'),
                        #    go.Scatter(x=dfpl.index, y=dfpl['ADX_neg'], line=dict(color='red', width=2), name="ADX-", yaxis='y3'),
                        #    go.Scatter(x=dfpl.index, y=dfpl['ADX_pos'], line=dict(color='green', width=2), name="ADX+", yaxis='y3'),
                           go.Scatter(x=dfpl.index, y=dfpl['RSI'], line=dict(color='black', width=2), name="RSI", yaxis='y4')]



    layout = go.Layout(title=f"Strategy for {df['symbol'][0]} found {len(df[df['Signal'] != ''])} signals", yaxis_title='Price', dragmode='pan',
                       xaxis=dict(rangeslider=dict(visible=False)),
                       yaxis=dict(fixedrange=False, domain=[0.3,1], anchor='y1'), 
                       yaxis2=dict(domain=[0.20,0.29], anchor='y2', fixedrange=True), 
                       yaxis3=dict(domain=[0.10,0.19], anchor='y3', fixedrange=False),
                       yaxis4=dict(domain=[0,0.09], anchor='y4', fixedrange=True))

    fig = go.Figure(data=data, layout=layout)

    # Configure x-axis to enable spikelines
    fig.update_xaxes(showspikes=True, spikecolor='gray', spikemode='across', spikesnap='cursor', spikedash='solid', spikethickness=1)

    # Configure y-axes to enable spikelines
    fig.update_yaxes(showspikes=True, spikecolor='gray', spikemode='across', spikesnap='cursor', spikedash='solid', spikethickness=1)


    pyo.plot(fig, filename='Strategy.html', auto_open=True)


### 4. Provide the signals for the most recent time frame

In [30]:
def get_current_signal_as_json(df):
    return df.last('1D').to_json(orient="records")

### 5. Wire up a backtesting function

In [34]:
from backtesting import Strategy, Backtest

# def bt_signal(values):
#     return values['Signal']

class Backtester(Strategy):

    def init(self):
        super().init()
        # self.bt_signal = self.I(bt_signal, self.data.df)

    def next(self):
        super().next()

        # Close out any long positions, in case there is strong upwards resistance
        if (self.position.is_long):
            if self.data.RSI[-1] > 70:
                self.position.close()

        # Close out any short positions, in case there is strong downards resistance
        if (self.position.is_short):
            if self.data.RSI[-1] < 30:
                self.position.close()

        # If we have no positions, let's check for signals
        if self.position.size == 0:

            if self.data.Signal[-1] == 'Buy':
            # print('Buy!')
                self.buy(size=self.data.Quantity[-1], 
                         limit=self.data.Limit[-1], 
                         sl=self.data.Stop_loss[-1], 
                         tp=self.data.Take_profit[-1])
                
            elif self.data.Signal[-1] == 'Sell':
            # print('Sell!')
        
                if (self.position.size == 0):
                    self.sell(size=self.data.Quantity[-1], 
                              limit=self.data.Limit[-1], 
                              sl=self.data.Stop_loss[-1],  
                              tp=self.data.Take_profit[-1])

cash = 10_000
commission = .00

def create_backtest(df):
    bt = Backtest(df, Backtester, cash=cash, commission=commission)
    return bt


### Testing it all so far

In [59]:
# Test the function
# f = open('../development_assets/AAPL_10y.json', 'r').read() # Win rate 66%
# f = open('../development_assets/AMZN_10y.json', 'r').read() ## Win rate 55%
# f = open('../development_assets/BRK-B_10y.json', 'r').read() ## Win rate 66%
# f = open('../development_assets/GOOG_10y.json', 'r').read() ## Win rate 85%
# f = open('../development_assets/JNJ_10y.json', 'r').read() ## Win rate 88%
# f = open('../development_assets/META_10y.json', 'r').read() ## Win rate 55%
# f = open('../development_assets/MSFT_10y.json', 'r').read() ## Win rate 88%
# f = open('../development_assets/NVDA_10y.json', 'r').read() ## Win rate 72%
# f = open('../development_assets/TSLA_10y.json', 'r').read() ## Win rate 0%
f = open('../development_assets/V_10y.json', 'r').read() ## Win rate 62%
df = parse_data(f)
df = add_indicators(df)
df = create_signals(df)
df = add_trades(df)
# get_current_signal_as_json(df)
# test = run_backtest(df)
fig = create_plot(df)

/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning:

invalid value encountered in scalar divide

/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning:

invalid value encountered in scalar divide



In [60]:
test=create_backtest(df)
stats = test.run()
test.plot()
stats

/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/backtesting/backtesting.py:912: UserWarning:

(2015-08-24 00:00:00+00:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119

/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'



Start                     2013-03-04 00:00...
End                       2023-03-01 00:00...
Duration                   3649 days 00:00:00
Exposure Time [%]                   26.102503
Equity Final [$]                 14314.009811
Equity Peak [$]                  17796.675884
Return [%]                          43.140098
Buy & Hold Return [%]              487.013639
Return (Ann.) [%]                     3.65606
Volatility (Ann.) [%]               12.823569
Sharpe Ratio                         0.285105
Sortino Ratio                        0.438403
Calmar Ratio                         0.186827
Max. Drawdown [%]                  -19.569194
Avg. Drawdown [%]                    -3.64695
Max. Drawdown Duration      748 days 00:00:00
Avg. Drawdown Duration       84 days 00:00:00
# Trades                                    8
Win Rate [%]                             62.5
Best Trade [%]                      24.775235
Worst Trade [%]                    -12.777436
Avg. Trade [%]                    

In [26]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,64,209,220,139.246083,162.752045,1504.381604,0.168809,2022-12-28 00:00:00+00:00,2023-01-13 00:00:00+00:00,16 days
